## Halloween Bonus

You wish to buy video games from the famous online video game store Mist.

Usually, all games are sold at the same price, $p$ dollars. However, they are planning to have the seasonal Halloween Sale next month in which you can buy games at a cheaper price. Specifically
* the first game will cost $p$ dollars;
* every subsequent game will cost $d$ dollars less than the previous one;
* this continues until the cost becomes less than or equal to $m$ dollars, after which every game will cost $m$ dollars. 

How many games can you buy during the Halloween Sale having a budget of $s$ dollars ?

**Input example:** $$\begin{cases}p=100\\ d=20\\ m=1\\ s=180\end{cases}$$

In [ ]:
import sys

def howManyGames(p, d, m, s):
  ans = 0
  while s >= p:
    s -= p
    ans += 1
    p = max(m, p - d)
  return ans

if __name__ == "__main__":
  p, d, m, s = map(int, input().strip().split(' '))
  answer = howManyGames(p, d, m, s)
  print(answer)

100 20 1 180
2


* If you are interested have a look at the [Halloween Strategy](https://www.investopedia.com/terms/h/halloween-strategy.asp).

In [1]:
import sys
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
sys.path.append('/content/drive/MyDrive/finance_course/2021/lesson5')

Mounted at /content/drive


## Interest Rate Derivatives

### Interest Rate Swaps

* Interest rate swaps (IRS) consist of a floating leg and a fixed leg. The contract parameters are:
  * start date $d_0$
  * notional $N$
  * fixed rate $K$
  * floating rate tenor (months)
  * maturity (years)


* **floating leg**: pays the reference EURIBOR fixing at a frequency equal to the index tenor:
    * so for example an IRS on a 3-month EURIBOR will pay a floating coupon every three months.


* **fixed leg**: pays a predetermined cash flow at annual frequency:
    * for simplicity we will only consider swaps with maturities which are multiples of 1 year.

In [2]:
# modify generate_dates
from datetime import date
from dateutil.relativedelta import relativedelta

def generate_dates(start_date, nmonths, tenor=12):
    dates = []
    for i in range(0, nmonths, tenor):
        dates.append(start_date + relativedelta(months=i))
    dates.append(start_date + relativedelta(months=nmonths))
    return dates

generate_dates(date.today(), 15, 4)

[datetime.date(2022, 3, 16),
 datetime.date(2022, 7, 16),
 datetime.date(2022, 11, 16),
 datetime.date(2023, 3, 16),
 datetime.date(2023, 6, 16)]

### Valuation

* Notation
    * $d_i^{\mathrm{fixed}}, d_i^{\mathrm{float}}$ be the fixed leg payment dates and floating leg payment dates;
    * $d$ the pricing date;
    * $D(d, d')$ the discount factor observed in date $d$ for the value date $d'$;
    * $F(d, d', d'')$ the forward rate observed in date $d$ for the period $[d', d'']$;  
    * $\tau = \frac{d_{j}^{\mathrm{float}}-d_{j-1}^{\mathrm{float}}}{360}$ the tenor.


* The NPV of the fixed leg is calculated as follows:

$$\mathrm{NPV}_{\mathrm{fixed}}(d, K) = N\cdot K\cdot\sum_{i=1}^{n}D(d, d_{i}^{\mathrm{fixed}})$$

* while the NPV of the floating leg is calculated as follows:

$$\mathrm{NPV}_{\mathrm{float}}(d, F) = N\cdot\sum_{i=1}^{m}F(d, d_{j-1}^{\mathrm{float}}, d_{j}^{\mathrm{float}}) \cdot \tau
\cdot D(d, d_{i}^{\mathrm{float}})$$

* Therefore the NPV of the swap (seen from the point of view of the counter-party which receives the floating leg) is

$$\mathrm{NPV}(d, F, K) = \mathrm{NPV}_{\mathrm{float}}(d,F) - \mathrm{NPV}_{\mathrm{fixed}}(d,K)$$

* It's actually more convenient to express the NPV of an IRS as a function of the fair value fixed rate $S$ of the IRS (**swap rate**)
    * $S$ is the value of $K$ which makes $\mathrm{NPV}=0$.

$$\mathrm{NPV}_{\mathrm{fixed}}(d, S) = \mathrm{NPV}_{\mathrm{float}}(d, F)\\[20pt]$$
$$N\cdot S\cdot\sum_{i=1}^{n}D(d, d_{i}^{\mathrm{fixed}}) = N\cdot\sum_{i=1}^{m}F(d, d_{j-1}^{\mathrm{float}}, d_{j}^{\mathrm{float}}) \cdot\tau\cdot D(d, d_{i}^{\mathrm{float}})\\[20pt]$$
$$S=\frac{\sum_{i=1}^{m}F(d, d_{j-1}^{\mathrm{float}}, d_{j}^{\mathrm{float}}) \cdot \tau\cdot D(d, d_{i}^{\mathrm{float}})}{\sum_{i=1}^{n}D(d, d_i^{\mathrm{fixed}})}\\[20pt]$$


* Once calculated $S$, we can express the $\mathrm{NPV}$ of an IRS as follows:

$$\begin{align}&\mathrm{NPV}(d, F, K) = \mathrm{NPV}_{\mathrm{float}}(d, F) - \mathrm{NPV}_{\mathrm{fixed}}(d, K) = & \\ \\ &= \underbrace{\mathrm{NPV}_{\mathrm{float}}(d, F) - \mathrm{NPV}_{\mathrm{fixed}}(d, S)}_{\mathrm{=\;0}} + \mathrm{NPV}_{\mathrm{fixed}}(d, S) - \mathrm{NPV}_{\mathrm{fixed}}(d, K) & \\ & = N\cdot(S-K)\cdot\underbrace{\sum_{i=1}^{n}D(d, d_{i}^{\mathrm{fixed}})}_{\mathrm{annuity}}\end{align}$$

In [10]:
# implement InterestRateSwap 
class InterestRateSwap:
  def __init__(self, nominal, start_date, fixed_rate, tenor_months, maturity_years):
    self.nominal = nominal
    self.fixed_rate = fixed_rate
    self.fixed_leg_dates = generate_dates(start_date, 12 * maturity_years)
    self.floating_leg_dates = generate_dates(start_date, 12 * maturity_years,
                                             tenor_months)
                
  def annuity(self, discount_curve):
    a = 0
    for i in range(1, len(self.fixed_leg_dates)):
      a += discount_curve.df(self.fixed_leg_dates[i])
    return a

  def npv(self, discount_curve, euribor_curve):
    S = self.swap_rate(discount_curve, euribor_curve)
    A = self.annuity(discount_curve)
  
    return self.nominal * (S - self.fixed_rate) * A

  def num(self, discount_curve, euribor_curve):
    s = 0
    for j in range(1, len(self.floating_leg_dates)):
        F = euribor_curve.forward_rate(self.floating_leg_dates[j], self.floating_leg_dates[j-1])
        tau = (self.floating_leg_dates[j] - self.floating_leg_dates[j-1]).days / 360
        D = discount_curve.df(self.floating_leg_dates[j])
        s += F * tau * D
    return s
    
  def swap_rate(self, discount_curve, euribor_curve):
    return self.num(discount_curve, euribor_curve) / self.annuity(discount_curve)

* The relevant inputs have been saved in [euribor_curve.xlsx](https://github.com/matteosan1/finance_course/raw/develop/libro/input_files/euribor_curve.xlsx?raw=true) and [discount_curve.xlsx](https://github.com/matteosan1/finance_course/raw/develop/libro/input_files/discount_curve.xlsx)

In [4]:
# import xlsx and define discount and euribor curves
import pandas as pd
from datetime import date
from finmarkets import DiscountCurve, ForwardRateCurve

pricing_date = date(2020, 10, 20)
discount_data = pd.read_excel('https://github.com/matteosan1/finance_course/raw/develop/libro/input_files/discount_curve.xlsx')
euribor_data = pd.read_excel('https://github.com/matteosan1/finance_course/raw/develop/libro/input_files/euribor_curve.xlsx')

dates = [pricing_date + relativedelta(months=i) for i in discount_data['delta']]
dc = DiscountCurve(dates, discount_data.loc[:, 'df'])

dates = [pricing_date + relativedelta(months=i) for i in euribor_data['delta']]
fr = ForwardRateCurve(dates, euribor_data.loc[:, 'rate'])

print(dc.df(date(2022, 1, 1)))

print(fr.forward_rate(date(2022, 1, 1), date(2022, 7, 1)))

1.0042742709951005
0.08839986314124657


* Test our class instantiating an IRS with 1M notional, fixed rate of 5%, 6 month tenor and a maturity of 4 years.

In [5]:
# test it
start_date = pricing_date + relativedelta(months=1)
irs = InterestRateSwap(1e6, start_date, 0.05, 6, 4)
print ("{:.2f}".format(irs.npv(dc, fr)))

170152.50


* Can you guess what could be the **swap rate** given the value obtained for the NPV ? 
    * in particular will it be higher or lower than the IRS fixed rate of 5% ?
    * (remember that we are looking at this contracts from the point of view of the receiver of the floating leg...)

In [ ]:
# check the swap rate
swap_rate = irs.swap_rate(dc, fr)
print (swap_rate)

0.09222994268864719


* To check if the we have computed correctly the swap rate we can instanciate a new IRS with fixed rate equal to the just calculated swap rate and print its NPV, it should come very close to 0.

In [ ]:
# check NPV of new IRS
irs_new = InterestRateSwap(1e6, start_date, swap_rate, 6, 4)
irs_new.npv(dc, fr)

0.0

## Interest Rate Swaptions

* Swaptions are the equivalent of European options for the interest rate markets
    * they give the option holder the right but not the obligation, at the exercise date $d_{ex}$, to enter into an Interest Rate Swap at a pre-determined fixed rate.
    * the option holder will only choose to do this if the NPV of the underlying swap at $d_{ex}$ is positive.
    
    
* From the IRS NPV expression in terms of the swap rate $S$ therefore, we can see that the payoff of the swaption is

$$N\cdot \mathrm{max}(0, S(d_{\mathrm{ex}}) - K)\cdot\sum D(d, d_i^{\mathrm{fixed}})$$

* The key issue is now to estimate $S(d_{\mathrm{ex}})$ in order to evaluate the payoff of a swaption. 


* This will be shown with two alternative approaches.

#### Evaluation through Black-Scholes formula

* Here we'll use a generalization of the Black-Scholes-Merton formula applied to swaptions:

$$\mathrm{NPV} = N\cdot A\cdot [S \Phi(d_+) - K\Phi(d_-)]$$

* where $\Phi$ represents the cumulative distribution function of the normal distribution

$$d_{\pm} = \frac{\mathrm{log}(\frac{S}{K}) \pm \frac{1}{2}\sigma^{2}T}{\sigma\sqrt{T}}\qquad(\sigma~\textrm{is the volatility of the swap rate})$$

$$A =\sum_{i=1}^{p}D(d, d_{i}^{\mathrm{fixed}})\qquad\mathrm{(annuity})$$


* Consider a swaption whose underlying 6M-IRS has a notional of 1M, fixed rate of 5%, a maturity of 4 years, and in addition a swap rate volatility of 7%.

In [11]:
# define function to compute swaption payoff
from math import log, sqrt
from scipy.stats import norm 

def swaptionPayoffBS(irs, dc, fr, sigma, T):
    N = irs.nominal
    K = irs.fixed_rate
    S = irs.swap_rate(dc, fr)
    A = irs.annuity(dc)
    print (A)
    print (S)
    dp = (log(S/K) + 0.5*sigma**2*T)/(sigma*sqrt(T))
    dm = (log(S/K) - 0.5*sigma**2*T)/(sigma*sqrt(T))
    return N*A*(S*norm.cdf(dp)-K*norm.cdf(dm))

observation_date = date(2020, 10, 20)#date.today()
start_date = observation_date + relativedelta(years=1)
irs = InterestRateSwap(1e6, start_date, 0.05, 6, 4)
exercise_date = start_date
T = (exercise_date - observation_date).days/365
sigma = 0.07
price_bs = swaptionPayoffBS(irs, dc, fr, sigma, T)
print ("{:.2f}".format(price_bs))

4.032413999999999
0.10337132226067916
215215.27


#### Evaluation through Monte-Carlo Simulation

* Here it is assumed that the swap rate $S$ evolves following a log normal process
    * its distribution at $d_{\mathrm{ex}}$ (exercise date) is $S(d_{\mathrm{ex}}) = S(d)\mathrm{exp}(-\frac{1}{2}\sigma^{2}T+\sigma\sqrt{T}\epsilon)$ where $\epsilon\approx\mathcal{N}(0,1)$;
    * notice that it is assumed that the *drift* rate in the evolution of the swap rate is zero.

* To simulate the swap rate we can:
    1. sample the normal distribution $\mathcal{N}(0, 1)$ to calculate a large number of scenarios for $S(d_{\mathrm{ex}})$;
    2. evaluate the underlying swap's NPV at the exercise date, and consequently the swaption's payoff, for each scenario;
    3. take the average of these values to get the final estimate.

Further the 95% confidence level for the swaption simulation can be calculated.

In [13]:
# implement MC swaption payoff
import numpy as np
from math import exp, sqrt
from numpy.random import normal, seed

seed(1)
n_scenarios = 100000
payoffs = []
S0 = irs.swap_rate(dc, fr)
for _ in range(n_scenarios):
    S = S0 * exp(-sigma**2/2*T + sigma*np.random.normal()*np.sqrt(T))
    payoff = irs.nominal*max(0, S - irs.fixed_rate)*irs.annuity(dc)
    payoffs.append(payoff)

price_mc = np.mean(payoffs)
interval = 1.96*np.std(payoffs)/sqrt(n_scenarios)
print ("{:.2f} +- {:.2f}".format(price_mc, interval))

215365.18 +- 180.84


## Credit curves

* Just like a discount curve is a way of representing the underlying interest rates (or equivalently discount factors) implicit in the market quotes of a collection of real-world interest rate products, **credit curves** are a way of representing survival probabilities implied by credit default swaps.

* Credit default swaps (CDS) are instruments whose value depends on the likelihood that a given company (the curve's issuer) will suffer a credit event over a given period.

* A **credit event** can be a default, the failure to make payments, the issuer entering into bankruptcy proceedings, or the occurence of other legal events...
    * We will call a credit event a *default*;
    * and talk about **non-default probabilities** or survival probabilities($P_{\textrm{sur}}$), i.e. the probability that the issuer will not suffer a credit event before a given date 
        * *non-default probability is a cumulative probability* since refers to a time period. 


### Conditional Probability

* Conditional probability answers to the question "how should you update probabilities of events when there is additional information available ?". 

#### Example

* Imagine a fair die is rolled: 
    * let $A$ be the event that the outcome is an odd number ($A={1,3,5}$); 
    * let $B$ be the event that the outcome is less than or equal to $3$ ($B={1,2,3}$). 
    1. What is the probability of $A$ ($P(A)$) ? 
    2. What is the probability of $A$ given $B$ ($P(A|B)$) ?

* Being a simple example we can compute the result by hand:

$$P(A) = \cfrac{|A|}{|S|} = \cfrac{|\{1,3,5\}|}{6} = \cfrac{1}{2}\qquad\textrm{(where $S$ is the entire sample space)}$$

* If $B$ has occurred, the outcome must be among $\{1,2,3\}$. 

* For $A$ to also happen the outcome must be in $A\cap B = \{1,3\}$. 
    * Since the die is fair, we argue that $P(A|B)$ must be equal to

$$P(A|B) = \cfrac{|A\cap B|}{|B|} = \cfrac{2}{3}$$

* To generalize, divide numerator and denominator by the entire space of the events $|S|$ hence:

$$P(A|B) = \cfrac{|A\cap B|}{|B|} = \cfrac{\cfrac{|A\cap B|}{|S|}}{\cfrac{|B|}{|S|}} = \cfrac{P(A\cap B)}{P(B)}$$

<img src="https://drive.google.com/uc?id=16bF8jTwIMLPPDK2vIP99CV-RcBJTocyk" width=500>

### Hazard Rate
* Hazard rate represents the instantaneous probability of a company defaulting *conditioned* on it not having defaulted until that moment.

* Hazard rate is often called a *conditional failure rate* since it's expression is a direct application of the conditional probability concept.

$$\lambda(t) = \lim_{dt\rightarrow 0} \cfrac{P_{\textrm{def}}(t, t+dt|\tau\gt t)}{dt} = \cfrac{dP_{\textrm{def}}(t)}{P_{\textrm{def}}(t, +\infty)}\cfrac{1}{dt} = \cfrac{d(1-P_{\textrm{sur}}(t))}{P_{\textrm{sur}}(t_0, t)}\cfrac{1}{dt} = -\cfrac{dP_{\textrm{sur}}}{dt}\cfrac{1}{P_{\textrm{sur}}(t_0, t)}$$

* where the default (survival) probability is indicated by $P_{\textrm{def}}$ ($P_{\textrm{sur}}$), the hazard rate by $\lambda$ and the time of default with $\tau$. The minus sign derives from the fact that $P_{\textrm{sur}}$ is a **non** default probability while the hazard rate is defined in terms of the probability of default $P_{\textrm{def}}$.

$$P_{\textrm{def}}(t, +\infty) = (1-P_{\textrm{def}}(t_0, t)) = (1-(1 - P_{\textrm{sur}}(t_0, t)) =  P_{\textrm{sur}}(t_0, t)$$

* Given the hazard rate the survival probability can be determined as:

$$\lambda(t) = -\cfrac{1}{dt}\cdot\cfrac{dP_{\textrm{sur}}}{P_{\textrm{sur}}} = -\cfrac{d(\textrm{log}P_{\textrm{sur}})}{dt}\\[5pt]$$

$$P_{\textrm{sur}}(t_0, t) = e^{-\int_{t_0}^{t}\lambda(s) ds}$$

* Remember the short rate, $r_t$, is the interest rate at which an entity can borrow money for an infinitesimally short period of time from $t$ to $t+dt$.

### CreditCurve class

In [14]:
# implement CreditCurve class
from numpy import interp
from dateutil.relativedelta import relativedelta

class CreditCurve:
    def __init__(self, pillar_dates, ndps):
        self.start_date = pillar_dates[0]
        self.pillar_days = [(pd - self.start_date).days for pd in pillar_dates]
        self.ndps = ndps
    
    def ndp(self, value_date):
        value_days = (value_date - self.start_date).days
        return interp(value_days, self.pillar_days, self.ndps)

    def hazard(self, value_date):
        ndp_1 = self.ndp(value_date)
        ndp_2 = self.ndp(value_date + relativedelta(days=1))
        delta_t = 1.0 / 365.0
        h = -1.0 / ndp_1 * (ndp_2 - ndp_1) / delta_t
        return h

* As usual we test the newly developed class with some data.

In [15]:
# set observation_date and CreditCurve
from datetime import date
from dateutil.relativedelta import relativedelta

start_date = date.today()
cc = CreditCurve(
    [start_date, start_date + relativedelta(years=2)],
    [1.0, 0.8])

In [16]:
# check ndp in 1 year
cc.ndp(start_date + relativedelta(years=1))

0.9001367989056088

In [17]:
# check hazard in 1 year
cc.hazard(start_date + relativedelta(years=1))

0.11094224924016097

## Credit Deafult Swaps

* A Credit Default Swap (CDS) is a financial swap agreement that the seller of the CDS will compensate the buyer in case of a credit event. 
    * The seller of the CDS insures the buyer against some reference asset defaulting. 
    * The buyer of the CDS makes a series of payments (the CDS "fee" or "spread") to the seller and, in exchange, may expect to receive a payoff if the asset defaults. 

* CDSs are made up of two legs:
    * the *default* leg: which pays $LGD = F(1 - R)$, known as the **loss given default**, if and when the credit event occurs, $F$ is the face value of the contact, $R$ is the recovery rate (usually set around 40%);
    * the *premium* leg: which pays the *spread* $S$ every m months until the credit event occurs.

### Premium Leg Valuation

Let's start with the premium leg. We will use the following notation:

* $d$ today's date;
* $d_0$ the start date of the CDS (could be different from $d$);
* $d_1, ..., d_n$ the payment dates of the premium leg, which occur at a m-month frequency (we assume that $d_n$ is the end date of the CDS);
* $D(d')$ the discount factor between $d$ and $d'$;
* $P_{\textrm{sur}}(d')$ the survival probability between $d$ and $d'$;
* $\tau$ the random variable representing the date of the credit event.

At each payment date $d_i$, a flow $F\cdot S$ is paid if and only if the credit event has not occurred before that date. Since the NPV depends on the default probability, the value of the premium leg can only be estimated by an expectation

$$f_{\textrm{premium}}^i = \mathbb{E}\left[F \times S \times D(d_i) \times \mathbb{1}(\tau > d_i) \right]$$

where $\mathbb{1}(\tau > d_i)$ means that the expectation value has to be evaluated when $\tau > d_i$. Remember that if $x$ is a random variable with a finite number of finite outcomes $x_{1},x_{2},\ldots ,x_{k}$ occurring with probabilities $p_{1},p_{2},\ldots ,p_{k}$ respectively, the expectation of $x$ is:

$$\mathbb{E}[x] = \sum _{i=1}^{k}x_{i}\,p_{i}=x_{1}p_{1}+x_{2}p_{2}+\cdots +x_{k}p_{k}$$
which is the weighted sum of the $x_i$, with $p_{i}$ values being the weights. 

In our case $x_i = F\cdot S\cdot D(d_i)$ and $p_i=P_{\textrm{sur}}(d_i)$ so the NPV of the leg can be expressed as:

$$\textrm{NPV}_{\textrm{premium}} = F\cdot S\cdot \sum_{i=1}^{n} D(d_i) \cdot P_{\textrm{sur}}(d_i)$$

### Default Leg Valuation

Let's assume the LGD is paid out on the same date on which the credit event occurs, although it can potentially be paid out on any date between $d_0$ and $d_n$. Mathematically, therefore, the NPV of the premium leg can be expressed as follows:

$$\mathrm{NPV_{default}} = \mathbb{E} \left[F(1-R) \times D(\tau) \times \mathbb{1}(\tau \leq d_n) \right] $$

Using the laws of probability, we can break this down into the sum of "daily NPVs" calculated as a function of the daily default probabilities $P_{\textrm{def}}$:

$$
\begin{align*}
\mathbb{E}\left[F(1-R) \times D(\tau) \times \mathbb{1}(\tau \leq d_n) \right]
&= \sum_{d'=d_0}^{d_n} \mathbb{E}[ F(1-R) \times D(\tau) | \tau = d'] P_{\textrm{def}}(\tau = d') \\
&= F(1-R) \sum_{d'=d_0}^{d_n} D(d') \left( P_{\textrm{def}}(\tau \geq d') - P_{\textrm{def}}(\tau \geq d'+1) \right) \\
&= F(1-R) \sum_{d'=d_0}^{d_n} D(d') \left( P_{\textrm{sur}}(d') - P_{\textrm{sur}}(d'+1) \right)
\end{align*}
$$

where the last step holds since $P_{\textrm{def}}(\tau\geq d') = 1 - P_{\textrm{def}}(\tau < d') = 1 - (1-P_{\textrm{sur}}(\tau < d')) = P_{\textrm{sur}}(\tau < d')$.

<img src="https://drive.google.com/uc?id=1biFm8CEp85XRqjTC-ePC0tOqCxNLkxHE">

In [18]:
# credit default swap class with breakeven method
#from finmarkets import generate_dates

class CreditDefaultSwap:
    def __init__(self, notional, start_date, fixed_spread,
                 maturity_y, tenor=3, recovery=0.4):
        self.notional = notional
        self.payment_dates = generate_dates(start_date,
                                            maturity_y*12, tenor)
        self.fixed_spread = fixed_spread
        self.recovery = recovery

    def npv_premium_leg(self, discount_curve, credit_curve):
        npv = 0
        for i in range(1, len(self.payment_dates)):
            npv += (self.fixed_spread *
                    discount_curve.df(self.payment_dates[i]) *
                    credit_curve.ndp(self.payment_dates[i]))
        return npv * self.notional

    def npv_default_leg(self, discount_curve, credit_curve):
        npv = 0
        d = self.payment_dates[0]
        while d < self.payment_dates[-1]:
            npv += discount_curve.df(d) * (
                   credit_curve.ndp(d) -
                   credit_curve.ndp(d + relativedelta(days=1)))
            d += relativedelta(days=1)
        return npv * self.notional * (1 - self.recovery)

    def npv(self, discount_curve, credit_curve):
        return self.npv_default_leg(discount_curve, credit_curve) - \
               self.npv_premium_leg(discount_curve, credit_curve)

* Below a simple test of the class, using [discount_curve.xlsx](https://github.com/matteosan1/finance_course/raw/develop/libro/input_files/discount_curve.xlsx?raw=true).

In [19]:
# test CDS class with previous inputs 
import pandas as pd
from finmarkets import DiscountCurve
from dateutil.relativedelta import relativedelta
from datetime import date

dc_data = pd.read_excel("https://github.com/matteosan1/finance_course/raw/develop/libro/input_files/discount_curve.xlsx")
pricing_date = date.today() 

dates = [pricing_date + relativedelta(months=i) for i in dc_data['delta']]
dc = DiscountCurve(dates, dc_data['df'])

credit_curve = CreditCurve([pricing_date,
                            pricing_date + relativedelta(months=36)],
                            [1.0, 0.7])
cds = CreditDefaultSwap(1e6, pricing_date, 0.03, 3)

In [ ]:
# check default leg, premium leg and npv
print ("{:.2f}".format(cds.npv_premium_leg(dc, credit_curve)))
print ("{:.2f}".format(cds.npv_default_leg(dc, credit_curve)))

302972.62
180901.87


In [20]:
print ("{:.2f}".format(cds.npv(dc, credit_curve)))

-122070.74


### Determine Default Probabilities from Bond Prices

* The price of a bond is directly linked to the credit rating of the issuer.
    * There is always an associated default risk: the borrower might not be able to repay the loan.
    * Bonds with low ratings (*junk bonds*) are sold at lower prices since riskier;
    * high ratings bonds (*investment-grade bonds*) are sold at higher prices.


* The average default intensity can be approximated by considering the spread $s$ between a risky bond yield and the risk-free rate, and $R$ the recovery rate. 
    * The expected loss due to default can be expressed both in terms of spread and loss given default:
    
$$s \Delta t \approx \lambda\Delta t (1 − R) \implies \bar{\lambda} = \cfrac{s}{(1-R)}$$


* Imagine for example a bond that yields 150 bp more than a similar risk-free bond and assume an expected recovery rate of 40%.

$$\bar{\lambda} = \cfrac{0.015}{(1-0.4)} = 2.5\%$$

In [35]:
from math import exp
import numpy as np

default_dates = [0.5, 1.5, 2.5, 3.5, 4.5]
R = 0.4
def bond_price(N, C, y, maturity, tenor):
  val = 0
  for i in np.arange(tenor, maturity+tenor, tenor):
    #print(i)
    val += N*C*tenor*exp(-y*i)
    #print (val)
  val += N*exp(-y*i)
  return val

def expected_loss(N, C, r, maturity, tenor):
  loss = 0
  for i in range(len(default_dates)):
    val = 0
    for t in np.arange(default_dates[i], maturity+tenor, tenor):
      val += N*C*tenor*exp(-r*(t - default_dates[i]))
    val += N * exp(-r*(t - default_dates[i]))
    val = (val - N*R)*exp(-r*default_dates[i])
    loss += val
  return loss

Pr = bond_price(100, 0.06, 0.07, 5, 0.5)
Prf = bond_price(100, 0.06, 0.05, 5, 0.5)
spread = abs(Pr - Prf)
el = expected_loss(100, 0.06, 0.05, 5, 0.5)

print ("{:.2f}%".format(spread / el*100))

3.03%


## Estimate Default Probabilities from CDS

* We can estimate default probabilities (hence credit curves) from CDS quotes using *bootstrap*:
    1. collect market quotes for a number of CDS with different maturities;
    2. create the corresponding CDS objects;
    3. define a credit curve whose pillars are the CDS maturity dates and the survival probabilities are unknown;
    4. define an objective function to minimize the sum of the squared CDS's NPVs;
    5. set the non-default probabilities to an initial value and define their range of variability between [0, 1] since they are probabilities and fix "today's" probability to 1 since there hasn't been any default;
    6. run the minimization.

* The file with a test set of market quotes is [cds_quotes.xlsx](https://github.com/matteosan1/finance_course/raw/develop/libro/input_files/cds_quotes.xlsx).


In [36]:
# bootstrapping for CDS
from scipy.optimize import minimize
import pandas as pd

pricing_date = date.today()
dc = pd.read_excel("https://github.com/matteosan1/finance_course/raw/develop/libro/input_files/discount_curve.xlsx")
mq = pd.read_excel("https://github.com/matteosan1/finance_course/raw/develop/libro/input_files/cds_quotes.xlsx")

dates = [pricing_date + relativedelta(months=i) for i in dc['delta']]
discount_curve = DiscountCurve(dates, dc['df'])

cdswaps = []
pillar_dates = [pricing_date]
for i in range(len(mq)):
    cds = CreditDefaultSwap(1e6, pricing_date,
                            mq.loc[i, 'quote'],
                            mq.loc[i, 'months']//12)
    cdswaps.append(cds)
    pillar_dates.append(cds.payment_dates[-1])

def objective_function(unknown_ndps):
    credit_curve = CreditCurve(pillar_dates, unknown_ndps)
    sum_sq = 0
    for cds in cdswaps:
        sum_sq += cds.npv(discount_curve, credit_curve)**2
    return sum_sq

ndp_guess = [1 for _ in range(len(pillar_dates))]
bounds = [(0.01, 1) for _ in range(len(pillar_dates))]
bounds[0] = (1, 1)

r = minimize(objective_function, ndp_guess, bounds=bounds)

In [ ]:
print (r)

      fun: 4.033649375510235e-05
 hess_inv: <7x7 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 3.68333637e+04, -4.54225191e+01, -3.67204963e+01,  6.93941518e+00,
        5.33677480e+01,  3.78568243e+01,  8.41352439e+01])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 296
      nit: 18
   status: 0
  success: True
        x: array([1.        , 0.90794474, 0.80382715, 0.70843915, 0.48031939,
       0.29174293, 0.06685517])


In [ ]:
cc = CreditCurve(pillar_dates, r.x)
for i in range(len(pillar_dates)):
    print ("S(t<{}): {:.2f}".format(pillar_dates[i], r.x[i]))

S(t<2021-10-28): 1.00
S(t<2022-10-28): 0.91
S(t<2023-10-28): 0.80
S(t<2024-10-28): 0.71
S(t<2027-10-28): 0.48
S(t<2031-10-28): 0.29
S(t<2041-10-28): 0.07
